In [1]:
import numpy as np
from math import sqrt
import pandas as pd

In [2]:
class UnlimitedDataWorks:

    def __init__(self, deg):
        self.exp = []
        for i in range(deg+1):
            for j in range(deg+1):
                if i+j <= deg:
                    self.exp.append((i, j))

    def train_test_split(self, dataframe):
        self.data = pd.DataFrame([])
        self.count = -1
        for (a, b) in self.exp:
            self.count += 1
            res = ((dataframe["lat"] ** b) * (dataframe["lon"] ** a))
            self.data.insert(self.count, "col" + str(a) + str(b), res, True)

        self.count += 1
        normalize = lambda x: ((x - x.min()) / (x.max() - x.min()))
        dataframe = normalize(dataframe)
        self.data = normalize(self.data)
        self.data["col00"] = [1.0]*len(self.data)
        
        # generate a 70-20-10 split on the data:
        X = self.data[:304113]
        Y = dataframe["alt"][:304113]
        xval = self.data[304113:391088]
        yval = dataframe["alt"][304113:391088]
        x = self.data[391088:]
        y = dataframe["alt"][391088:]   
        return (X, Y, xval, yval, x, y)

In [3]:
class RegressionModel:

    def __init__(self, N, X, Y, x, y, xval, yval):
        """
        X :: training data                  (304113 x 3)
        x :: testing data                   (43786 x 3)
        Y :: training target values         (304113 x 1)
        y :: testing target values          (43786 x 1)
        xval :: validation data             (86975 x 3)
        yval :: validation training data    (86975 X 1)
        """
        self.N = N
        self.X = np.array(X)
        self.Y = np.array(Y)
        self.x = np.array(x)
        self.y = np.array(y)
        self.xval = np.array(xval)
        self.yval = np.array(yval)

    def score(self, weights):
        """
        the following method helps us find the
        R2 (R-squared) error of a given training data
        wrt the generated weights
        """
        ss_tot = sum(np.square(np.mean(self.y) - self.y))
        ss_res = sum(np.square((self.x @ weights) - self.y))
        rmse = sqrt(ss_res/len(self.x))
        r2 = (1-(ss_res / ss_tot))
        return [r2*100, rmse]

    def gradient_descent(self):
        """
        train till error is almost constant
        """
        lr = 8.75e-7
        prev_err, count = 1e10, 0
        W = np.random.randn(self.N)
        while True:
            diff = ((self.X @ W) - self.Y)
            err = 0.5 * (diff @ diff)
            grad = 0.5 * (self.X.T @ diff)
            if count % 500 == 0:
                print("epoch =", count, "| err_diff =", prev_err-err)
                print("error = ", err, "||", W)
                print("score =", self.score(W), end="\n\n")
            W -= lr * grad
            if abs(prev_err-err) <= 5e-4:
                break
            prev_err = err
            count += 1
        print(count, err)
        print(W, self.score(W), end="\n\n")

    def gradient_descent_L1_reg(self):
        """
        attempts a L1 regularization on the data
        considering 10% of training data as validation data
        """
        W_fin = np.array([])
        lr, l1_fin = 8e-7, 0
        MVLE = 1e10
        L1_vals = np.linspace(0.0, 1.0, 11)
        sgn = lambda x: (x / abs(x))
        for l1 in L1_vals:
            prev_err, count = 1e10, 0
            W = np.random.randn(self.N)
            while True:
                diff = ((self.X @ W) - self.Y)
                err = 0.5 * ((diff @ diff) + l1*sum([abs(w) for w in W]))
                if count % 500 == 0:
                    print("L1 hyperparamter =", l1, end=", ")
                    print("epoch =", count, "| err_diff =", prev_err-err)
                    print("error = ", err, "||", W)
                    print("score =", self.score(W), end="\n\n")
                sgn_w = np.array([sgn(w) for w in W])
                W -= lr * ((self.X.T @ diff) + 0.5*l1*sgn_w)
                if abs(prev_err-err) <= 0.01:
                    break
                prev_err = err
                count += 1
            VLD = ((self.xval @ W) - self.yval)
            VLE = 0.5 * ((VLD.T @ VLD) + l1*sum([abs(w) for w in W]))
            if VLE < MVLE:
                W_fin = W
                l1_fin = l1
                MVLE = VLE
        print(MVLE, l1_fin, W_fin)

    def gradient_descent_L2_reg(self):
        """
        attempts a L2 regularization on the data
        considering 10% of training data as validation data
        """
        W_fin = np.array([])
        lr, l2_fin = 5e-7, 0
        MVLE = 1e10
        L2_vals = np.linspace(0.0, 1.0, 11)
        for l2 in L2_vals:
            prev_err, count = 1e10, 0
            W = np.random.randn(self.N)
            while True:
                diff = ((self.X @ W) - self.Y)
                err = 0.5 * ((diff @ diff) + l2*sum([w*w for w in W]))
                if count % 500 == 0:
                    print("L2 hyperparamter =", l2, end=", ")
                    print("epoch =", count, "| err_diff =", prev_err-err)
                    print("error = ", err, "||", W)
                    print("score =", self.score(W), end="\n\n")
                W -= lr * ((self.X.T @ diff) + l2*W)
                if abs(prev_err-err) <= 0.01:
                    break
                prev_err = err
                count += 1
            VLD = ((self.xval @ W) - self.yval)
            VLE = 0.5 * ((VLD.T @ VLD) + l2 * (W.T @ W))
            if VLE < MVLE:
                W_fin = W
                l2_fin = l2
                MVLE = VLE
        print(MVLE, l2_fin, W_fin)

    def fit(self):
        """
        solves for optimal weights using system of
        N linear equations; AW = B, hence, W = inv(A)*B
        """
        B = self.X.T @ self.Y
        A = self.X.T @ self.X
        W = (np.linalg.inv(A)) @ B
        print(W, self.score(W))
        tmp = ((self.X @ W) - self.Y)
        print("train_error =", 0.5 * (tmp @ tmp))

In [4]:
columns = ["junk", "lat", "lon", "alt"]
raw_df = pd.read_csv("3D_spatial_network.txt", sep=',', header=None,
                     names=columns).drop("junk", 1)

deg = input("Enter the Degree of the Polynomial:")
pre_processor = UnlimitedDataWorks(deg=int(deg))
X_train, Y_train, x_val, y_val, x_test, y_test = pre_processor.train_test_split(raw_df)

model = RegressionModel(N=pre_processor.count,
                        X=X_train,
                        Y=Y_train,
                        x=x_test,
                        y=y_test,
                        xval=x_val,
                        yval=y_val)

model.fit()
model.gradient_descent()
# model.gradient_descent_L1_reg()
# model.gradient_descent_L2_reg()

Enter the Degree of the Polynomial:6
[ 1.28157283e+02  4.25047105e+05 -2.48499973e+06  5.29818476e+06
 -5.86510895e+06  3.26349377e+06 -6.24052203e+05  4.39269199e+05
  1.25620547e+04  3.41928625e+05 -1.64085375e+05 -1.22191844e+05
 -2.19000031e+05 -7.32193547e+05  1.25374592e+05 -2.97474281e+05
  5.96004469e+05  7.33931250e+03 -1.07784406e+05 -3.99701152e+05
 -2.96484266e+05  1.89696812e+05  3.19045656e+05  4.81178453e+05
 -2.27818461e+05  4.16299938e+05 -3.25788516e+04 -3.46609850e+05] [-57.15350550914704, 0.14637633170454326]
train_error = 2886.0601486641854
epoch = 0 | err_diff = 9999340007.56213
error =  659992.4378699014 || [ 0.17139107 -0.34021537 -0.30301937  1.16708612  0.07123813 -0.50763413
  0.26731105  0.5664501  -0.60055334 -0.48231611  0.06966841 -0.63224193
  0.2275899  -0.17062913  1.35600235  0.26219095  0.06626893 -2.22402218
  1.35395193 -0.31663364 -0.18061404  1.0128096  -1.47593055 -1.82005334
  0.56233162 -0.72919256 -0.72266932 -0.45917789]
score = [-33225.9955

epoch = 8000 | err_diff = 0.0007926925154606579
error =  2512.310962595536 || [ 0.14093455 -0.0766818  -0.10884255  1.28436477  0.10544507 -0.56113549
  0.1230829   0.71704348 -0.33549345 -0.28504214  0.19053463 -0.59424435
  0.17796038 -0.01241955  1.62341879  0.46293698  0.19095597 -2.1820741
  1.51971617 -0.04609327  0.02395442  1.14153675 -1.30267386 -1.54570115
  0.77105427 -0.54850627 -0.44388681 -0.27112547]
score = [7.570294566093394, 0.11225730383413503]

epoch = 8500 | err_diff = 0.0006339003821267397
error =  2511.956275642471 || [ 0.13983893 -0.07595139 -0.1084038   1.28403872  0.10390247 -0.56430891
  0.11791491  0.7154942  -0.33452952 -0.28429953  0.19053709 -0.59544804
  0.17512889 -0.01346047  1.62466005  0.46400353  0.19129941 -2.18293034
  1.51917992 -0.04453552  0.02536368  1.14223301 -1.30270917 -1.54379189
  0.77282389 -0.54804441 -0.44159465 -0.27017026]
score = [7.625713291361313, 0.11222364533957116]

epoch = 9000 | err_diff = 0.0005144552133060643
error =  2511